### **Implementasi**

In [1]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import re
#stopwords
import nltk
from nltk.corpus import stopwords

#stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#spit data
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/PPW/tugas

/content/drive/MyDrive/PPW/tugas


In [ ]:
df = pd.read_csv("Tugas-Crawling-Data-Berita-2-kategori.csv")

In [ ]:
columns_to_drop = ['NO', 'Judul Berita', 'Isi Berita', 'Tanggal Berita', 'Kategori Berita']
data_cleaned = df.drop(columns=columns_to_drop, errors='ignore')

# Menampilkan data yang sudah dibersihkan
print(data_cleaned.head())

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]


In [ ]:
df['Kategori'] = df['Kategori Berita'].map({'Politik': 0, 'Gaya Hidup': 1})

In [ ]:
def remove_url(data_berita):
  url = re.compile(r'https?://\S+|www\.S+')
  return url.sub(r'', data_berita)

def remove_html(data_berita):
  html = re.compile(r'<.#?>')
  return html.sub(r'', data_berita)

def remove_emoji(data_berita):
  emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"
      u"\U0001F300-\U0001F5FF"
      u"\U0001F680-\U0001F6FF"
      u"\U0001F1E0-\U0001F1FF""]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', data_berita)

def remove_numbers(data_berita):
  data_berita = re.sub(r'\d+', '', data_berita)
  return data_berita

def remove_symbols(data_berita):
  data_berita = re.sub(r'[^a-zA-Z0-9\s]', '', data_berita)
  return data_berita

df['cleansing'] = df['Isi Berita'].apply(lambda x: remove_url(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_html(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_emoji(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_symbols(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_numbers(x))

In [ ]:
def case_folding(text):
    if isinstance(text, str):
      lowercase_text = text.lower()
      return lowercase_text
    else :
      return text

df ['case_folding'] = df['cleansing'].apply(case_folding)

In [ ]:
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenize'] = df['case_folding'].apply(tokenize)

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

def remove_stopwords(text):
  return [word for word in text if word not in stop_words]

df['stopword_removal'] = df['tokenize'].apply(lambda x: ' '.join(remove_stopwords(x)))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming
def stemming(text):
    return [stemmer.stem(word) for word in text]

# Terapkan stemming setelah stopwords removal
df['stemming'] = df['stopword_removal'].apply(lambda x: ' '.join(stemming(x.split())))

# Menyimpan hasil stemming ke dalam file CSV
df.to_csv('/content/drive/MyDrive/PPW/tugas/stemming_output.csv', index=False)

In [ ]:
x = df['stemming'].values
y = df['Kategori'].values

In [ ]:
Xtrain, Xtest,Ytrain,Ytest = train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
vect = TfidfVectorizer()
X = vect.fit_transform(Xtrain)

X_array = vect.transform(Xtrain)

X_array.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.0855273 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.0855273 ],
       [0.09296982, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.11666782, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.010861  , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vect, f)

In [ ]:
# Membuat model Logistic Regression
model = LogisticRegression()

# Melatih model dengan data pelatihan
model.fit(X, Ytrain)

LogisticRegression()

In [ ]:
# Meyimpan model
with open('logistic_model', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# Load TF-IDF dan model logistic regression dari file pickle
with open("tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

In [ ]:
with open("logistic_model", "rb") as f:
    model = pickle.load(f)

In [ ]:
# Transformasi data baru dengan TF-IDF yang sudah dilatih
X_new = tfidf_vectorizer.transform(Xtest)

# Lakukan prediksi dengan model logistic regression
prediksi = model.predict(X_new)

print("Prediksi:", prediksi)

Prediksi: [1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1]


In [ ]:
accuracy = accuracy_score(Ytest, prediksi)
print(f"Akurasi: {accuracy * 100:.2f}%")

Akurasi: 100.00%


In [ ]:
# Load TF-IDF dan model logistic regression
with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('logistic_model', 'rb') as f:
    logistic_model = pickle.load(f)

In [ ]:
# -memPreprosesing input pengguna
def preprocess_input(text):
    text = remove_url(text)
    text = remove_html(text)
    text = remove_emoji(text)
    text = remove_symbols(text)
    text = remove_numbers(text)
    text = case_folding(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)
    return ' '.join(tokens)

In [ ]:
# Untuk memprediksi kategori dari input pengguna
def predict_category(user_input):
    # Preprocessing input
    preprocessed_input = preprocess_input(user_input)

    # Transformasi input pengguna menggunakan TF-IDF
    input_tfidf = tfidf_vectorizer.transform([preprocessed_input])

    # Memprediksi Kategori menggunakan MODEL LOGISTIC REGRESSION
    prediction = logistic_model.predict(input_tfidf)

    # Memetakan presiksi kategori
    category_map = {0: 'Politik', 1: 'Gaya Hidup'}
    return category_map[prediction[0]]

In [ ]:
# input pengguna
user_input = input("Masukkan isi berita: ")
predicted_category = predict_category(user_input)
print(f"Kategori berita: {predicted_category}")

Masukkan isi berita: Menjadi tugas dan tanggung jawab kita bersama, seluruh anggota DPR RI, untuk dapat menjalankan amanat rakyat dengan sebaik-baiknya," ujar Puan di hadapan para anggota dewan.  Puan menjelaskan bahwa setiap kali pemilu selesai dan rakyat telah memilih wakilnya, selalu ada harapan bahwa kekuasaan yang diberikan kepada DPR RI bisa membawa perubahan positif bagi kesejahteraan rakyat.  Namun, menurut Puan, Indonesia dihadapkan pada berbagai tantangan yang kompleks dalam upaya membangun negeri ini. Beberapa masalah struktural yang menjadi sorotan adalah kualitas sumber daya manusia (SDM), kedaulatan pangan, energi, pengembangan industri nasional, serta pemberdayaan usaha mikro, kecil, dan menengah (UMKM).  "Kita juga menghadapi ketidakpastian ekonomi global dan geopolitik, serta perekonomian nasional yang masih dalam proses pemulihan," ujarnya.  Lebih lanjut, Puan menekankan bahwa DPR RI harus menggunakan fungsi-fungsi konstitusionalnya, seperti legislasi, anggaran, dan p